In [1]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

In [3]:
from qazure import get_llm, get_embedder

In [4]:
import logging
import sys

In [5]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
llm = get_llm()
embed_model = get_embedder()

In [7]:
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

In [9]:
# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)

In [10]:
agent_response = await agent.run("What is 1234 * 4567?")

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


In [11]:
str(agent_response)

'1234 multiplied by 4567 equals 5,635,678.'

In [ ]:
# Simple query by indexing
# TODO: Try multimedia inputs like mp3 and mp4
documents = SimpleDirectoryReader(
    input_files=["feedback.docx"]
).load_data()
feedback_index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


In [18]:
retrieval_tool = QueryEngineTool.from_defaults(
    query_engine=feedback_index.as_query_engine(),
    description="Use this retrieval tool to get information from feedback form.",
)

In [22]:
# Create an agent workflow with our calculator tool
retrieval_agent = FunctionAgent(
    tools=[retrieval_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that retrieves information using the retrieval tool",
)

In [25]:
ra_response = await retrieval_agent.run("Give me the names of the interviewer and the candidate.")
ra_response

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.op

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The name of the candidate is Sohel, and the interviewer is Akshay.')]), tool_calls=[ToolCallResult(tool_name='query_engine_tool', tool_kwargs={'input': 'Provide the names of the interviewer and the candidate.'}, tool_id='call_h7M9DQpb1ok5PBVEJ1Ck5dLb', tool_output=ToolOutput(content='The name of the candidate is Sohel, and the interviewer is Akshay.', tool_name='query_engine_tool', raw_input={'input': 'Provide the names of the interviewer and the candidate.'}, raw_output=Response(response='The name of the candidate is Sohel, and the interviewer is Akshay.', source_nodes=[NodeWithScore(node=TextNode(id_='9c91ad8c-0631-4d31-ac47-16625cc213f8', embedding=None, metadata={'file_name': 'feedback.docx', 'file_path': 'feedback.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 18225, 'creation_date'

In [27]:
# Create an agent workflow with our calculator tool
compound_agent = FunctionAgent(
    tools=[retrieval_tool,multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that answers queries.",
)

In [32]:
ca_response = await compound_agent.run("Give me the names of the interviewer and the candidate.")
print('Retrieval response:', ca_response)
ca_response = await compound_agent.run("What is 1234 * 4567?")
print('Multiply response:', ca_response)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.op